In [ ]:
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import re
import timeit
import datetime
import xgboost as xgb
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.graph_objs as go
import folium
import json
import utm
import scipy
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import joblib
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import folium
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import datetime
import scipy.stats as stats
import plotly.express as px
pd.set_option('mode.chained_assignment', None)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

# libraries for import all file names
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression


## Analisis Anomalies

In [ ]:
def calcularAnomaly(df, col, cont=0.005):
    #Crear una versio horaria del dataset
    df['timestamp'] = pd.to_datetime(df.index)
    df = df.set_index('timestamp').resample("H").mean().reset_index()
    
    #Usem el metode de Isolation Forest per trobar anomalies a les precipitacions
    output = df[[col, 'timestamp']]
    temp = df[[col, 'timestamp']]
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=cont,max_features=1.0)
    model.fit(temp[[col]].values)
    output['outliers']=pd.Series(model.predict(temp[[col]].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )
    output.rename(columns={col:'data'}, inplace=True)
    
    return output

In [ ]:
# Carregar dades pluja
def loadDataPrecipitacions():
    precipitacions = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
    return precipitacions

In [ ]:
# Carregar dades Alt Ter
def loadDataAltTer():
    altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
    santJoan = calcularAnomaly(altTer, "L17167-72-00001")
    massiesRoda = calcularAnomaly(altTer, "L08116-72-00002")
    
    return santJoan, massiesRoda

In [ ]:
# Carregar dades Baix Ter
def loadDataBaixTer():
    baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)
    pasteralCabal = calcularAnomaly(baixTer, "F001242")
    colomers = calcularAnomaly(baixTer, "L17055-72-00002")
    torroellaMontegri = calcularAnomaly(baixTer, "L17199-72-00001")
    
    return pasteralCabal, colomers, torroellaMontegri

In [ ]:
#Carregar dades precipitacions Alt Ter:
def loadDataPrecipitacionsAlt():
    precipitacions = pd.read_csv("finalsDF/DF_SMC.csv", index_col=0)
    #Carregar dades
    santPau = calcularAnomaly(precipitacions, "CI")
    gurb = calcularAnomaly(precipitacions, "V3")
    
    #Moving Average
    santPau['data'] = santPau['data'].rolling(window=5).mean()
    gurb['data'] = santPau['data'].rolling(window=5).mean()
    
    return santPau, gurb

In [ ]:
def loadDataPrecipitacionsBaix():
    precipitacions = pd.read_csv("finalsDF/DF_SMC.csv", index_col=0)
    #Carregar dades
    angles = calcularAnomaly(precipitacions, "DN")
    talladaEmporda = calcularAnomaly(precipitacions, "UB")
    
    #Moving Average
    angles['data'] = angles['data'].rolling(window=5).mean()
    talladaEmporda['data'] = talladaEmporda['data'].rolling(window=5).mean()
    
    return angles, talladaEmporda

In [ ]:
def selectTimespan(df, start_date, end_date):
    df = df.loc[(df['timestamp'] >= start_date) &  (df['timestamp'] <= end_date)].copy(deep=True)
    return df

In [ ]:
# Carregar dades rius

alt1, alt2 = loadDataAltTer()
baix1, baix2, baix3 = loadDataBaixTer()


In [ ]:
# Carregar dades precipitacions
altPrep1, altPrep2 = loadDataPrecipitacionsAlt()
baixPrep1, baixPrep2 = loadDataPrecipitacionsBaix()

# Periode Temporal

### Cerca de Periodes Temporals amb Anomalies (Alt Ter)

In [ ]:
fig= px.scatter(alt1, x='timestamp', y='data', color='outliers',title='Anomalies Sant Joan')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(alt2, x='timestamp', y='data', color='outliers',title='Anomalies Massies Roda')
fig2.update_xaxes(
    rangeslider_visible=True,
)

fig.show()
fig2.show()

Trobem anomalies a les dues estacions del Alt Ter a les següents dates:
- 2010    
    - 3 - 4 Maig 2010 (42-165 l/m3)
    - 9 - 13 Octubre 2010 (132 - 335 l/m3)
- 2011
    - 12 - 18 Març 2011 (161 - 541 l/m3) (https://www.elpuntavui.cat/territori/article/11-mediambient/384345-el-riu-ter-sendu-un-tram-del-carril-bici-a-salt-i-un-passallis-a-bescano.html)
    - 3 - 24 Novembre 2011 (44 - 208 l/m3)
- 2013
    - 5 - 8 Març 2013 (137 - 361 l/m3) (https://www.garrotxadigital.cat/2013/03/06/el-riu-ter-multiplica-per-cent-el-seu-cabal/)
    - 16 - 19 Novembre 2013 (43 - 174 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/694602-proteccio-civil-alerta-de-laugment-del-cabal-dels-rius-besos-ter-i-fluvia.html)
- 2014
    - 28 Novembre - 4 Desembre 2014 (204 - 1102 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/800291-situacio-dralerta-al-ter.html)
- 2015
    - 2 - 5 Novembre 2015 (162 - 718 l/m3) (https://www.ripollesdigital.cat/noticia/65872/el-cabal-del-riu-ter-al-seu-pas-per-ripoll-sacosta-als-400-metres-cubics-per-segon-aquesta)
- 2017
    - 12 - 26 Febrer 2017 (76 - 219 l/m3)
- 2018
    - 8 - 13 Abril 2018 (95 - 341 l/m3) (https://www.elter.net/noticia/22295/les-pluges-multipliquen-21-cabal-del-riu-ter-osona)
    - 14 - 20 Octubre 2018 (359 - 1573 l/m3) (https://www.naciodigital.cat/noticia/164869/fotos-videos-ter-es-desborda-al-seu-pas-osona) (Huracà Leslie)
- 2019
    - 21 - 25 Octubre 2019 (108 - 315 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/1684802-el-riu-ter-baixa-molt-ple-al-seu-pas-per-roda-de-ter.html)
- 2020
    - 20 - 27 Gener 2020 (162 - 1183 l/m3) (https://www.consorcidelter.cat/actualitat/noticies/2020/01/27/gloria-al-ter) (Temporal Glòria)
    - 7 - 21 Juny 2020 (44 - 242 l/m3)
    - 28 - 31 Agost 2020 (44 - 315 l/m3)
    - 26 - 30 November 2020 (57 - 250 l/m3) (https://www.ccma.cat/324/activats-el-protocols-per-confinar-els-veins-que-viuen-prop-de-la-llera-del-riu-ter/noticia/2983502/)
    

### Cerca de Periodes Temporals amb Anomalies (Baix Ter)

In [ ]:
fig= px.scatter(baix1, x='timestamp', y='data', color='outliers',title='Anomalies Pasteral - Cabal')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(baix2, x='timestamp', y='data', color='outliers',title='Anomalies Colomers')
fig2.update_xaxes(
    rangeslider_visible=True,
)
fig3 = px.scatter(baix3, x='timestamp', y='data', color='outliers',title='Anomalies Torroella - Montegrí')
fig3.update_xaxes(
    rangeslider_visible=True,
)

fig.show()
fig2.show()
fig3.show()

Trobem anomalies a les dues estacions del Alt Ter a les següents dates:
- 2011
    - 12 - 21 Març 2011 (357 - 544 - 245 l/m3) (També anomalia a part alta)
    - 13 - 26 Novembre 2011 (209 - 684 - 395 l/m3) (També anomalia a part alta)
- 2013
    - 4 - 9 Març 2013 (681 - 342 l/m3) (NOMËS A COLOMERS I TORROELLA - MONTEGRÍ) (També anomalía a part alta)
- 2014
    - 29 Novembre - 17 Decembre 2014 (286 - 1131 - 418 l/m3) (http://www.elter.net/noticia/17337/societat/manlleu/forta-crescuda-del-riu-ter-arrossega-vehicle-malars-sense-ferits) (Coincideix amb anomalia a la part alta del riu a finals de Novembre)
- 2018
    - 13 - 23 Octubre 2018 (248 - 523 - 269 l/m3) (Coincideix amb anomalia a la part alta)
    - 14 - 20 Novembre 2018 (821 - 490 l/m3) (NOMËS A COLOMERS I TORROELLA - MONTEGRÍ) (Coincideix amb anomalia a la part alta del riu a finals de Octubre)
- 2020
    - 19 Gener - 1 Febrer 2020 (1202 - 1147 - 863 l/m3) (Temporal Gloria, coincideix amb anomalia a la part alta del riu)
    - 17 - 26 Abril 2020 (763 - 387 l/m3) (NOMES A COLOMERS I TOROELLA - MONTEGRÍ) (Petita pujada a Massies Roda)

# Analisis Cabal

## Pearson

Ordre rius

- Alt Ter: Sant Joan de les Abadesses (L17167-72-00001) > Massies de Roda (L08116-72-00002)

- Baix Ter: Pasteral-Cabal (F001242) > Colomers (L17055-72-00002) > Torroella de Montgrí (L17199-72-00001)

In [ ]:
def pearson_forward_lag(x, y, debug=False, n=50,m=40, p_ini=0):
    timeList = []
    rList = []
    pList = []
    offset = 0
    time = "00:00"
    timeStr = "%H:%M"
    time = datetime.datetime.strptime(time, timeStr)
    while offset < m:
        r, p = stats.pearsonr(x[p_ini:n], y[p_ini+offset:offset+n])
        if debug == True:
            print(f"Time Lag: +{time.strftime('%H:%M')}h , Pearson r: {r} and p-value: {p}")
        timeList.append(time)
        rList.append(r)
        pList.append(p)
        time = time + timedelta(minutes=30)
        offset += 1
        
    return timeList, rList, pList

In [ ]:
def pearsonDF(inici, final, s1, s2):
    s1 = selectTimespan(alt1, inici, final)
    s2 = selectTimespan(alt2, inici, final)
    
    t, r, p = pearson_forward_lag(s1['data'], s2['data'])
    d = {'r':r, 'p':p}
    df = pd.DataFrame(data=d,index=t)
    df.index = df.index.strftime('%H:%M')
    
    temps = df.loc[(df['p'] < 0.05)]['r'].idxmax()
    valor = df.loc[(df['p'] < 0.05)]['r'].max()

    return {temps:valor}
    
    
    
    

In [ ]:
def pearsonTop(inici, final, s1, s2, n=5):
    s1 = selectTimespan(alt1, inici, final)
    s2 = selectTimespan(alt2, inici, final)
    
    t, r, p = pearson_forward_lag(s1['data'], s2['data'])
    d = {'r':r, 'p':p}
    df = pd.DataFrame(data=d,index=t)
    df.index = df.index.strftime('%H:%M')
    
    df = df.loc[(df['p'] < 0.05) & (df['r'] > 0.5)]
    df = df.sort_values(by=['r'])
    df.head(n)

    return df

In [ ]:
def subplotPearsonPerData(inici, final, alt1, alt2, baix1, baix2, baix3):
    a1 = selectTimespan(alt1, inici, final)
    a2 = selectTimespan(alt2, inici, final)
    b1 = selectTimespan(baix1, inici, final)
    b2 = selectTimespan(baix2, inici, final)
    b3 = selectTimespan(baix3, inici, final)
    
    
    
    t, r, p = pearson_forward_lag(a1['data'], a2['data'])
    d = {'r':r, 'p':p}
    df = pd.DataFrame(data=d,index=t)
    df.index = df.index.strftime('%H:%M')
    
    fig = px.line(df, x=df.index, y=["r", "p"], markers=True, title="Tram Alt1-Alt2")
    
    t, r, p = pearson_forward_lag(b1['data'], b2['data'])
    
    d = {'r':r, 'p':p}
    df2 = pd.DataFrame(data=d,index=t)
    df2.index = df2.index.strftime('%H:%M')
    
    fig2 = px.line(df2, x=df2.index, y=["r", "p"], markers=True, title="Tram Baix1-Baix2")
    
    t, r, p = pearson_forward_lag(b2['data'], b3['data'])
    
    d = {'r':r, 'p':p}
    df3 = pd.DataFrame(data=d,index=t)
    df3.index = df3.index.strftime('%H:%M')
    
    fig3 = px.line(df3, x=df3.index, y=["r", "p"], markers=True, title="Tram Baix2-Baix3")
    
    fig.show()
    fig2.show()
    fig3.show()

    
    

In [ ]:
#subplotPearsonPerData("01-10-2018","31-10-2018",alt1, alt2, baix1, baix2, baix3)
pearsonTop("01-03-2011","31-03-2011", alt1, alt2)

In [ ]:
significativeDates = [("01-03-2011","31-03-2011"),("01-11-2011","30-11-2011"),("01-03-2013","31-03-2013"),
                    ("01-04-2014","30-04-2014"),("01-10-2018","31-10-2018"),("01-11-2018","30-11-2018"),
                     ("01-01-2020","31-01-2020"),("01-04-2020","30-04-2020")]

for x in significativeDates: 
    print(x[0] + ", " + x[1] + ": " ,pearsonDF(x[0],x[1],alt1, alt2))

# Analisis Precipitacions (Glories)

In [ ]:
#Seleccionem el periode temporal (en aquesta part el temporal Gloria)

start_date = datetime.datetime.strptime("20/01/2020", "%d/%m/%Y")
end_date = datetime.datetime.strptime("26/01/2020", "%d/%m/%Y")

altPrep1 = selectTimespan(altPrep1, start_date, end_date)
altPrep2 = selectTimespan(altPrep2, start_date, end_date)
baixPrep1 = selectTimespan(baixPrep1, start_date, end_date)
baixPrep2 = selectTimespan(baixPrep2, start_date, end_date)

In [ ]:
#Coincidim les timeseries amb les dades dels rius
#alt1['precipitacions'] = altPrep1['data']
#alt2['precipitacions'] = altPrep2['data']
#baix1['precipitacions'] = baixPrep1['data']
#baix2['precipitacions'] = baixPrep2['data']

## Analisis Precipitacions Alt Ter

In [ ]:
fig = px.line(alt1, x="timestamp", y="data", markers=True, title='Cabal Sant Joan')
fig2 = px.line(altPrep1, x="timestamp", y="data", markers=True, title="Precipitations Sant Joan")
fig.show()
fig2.show()

In [ ]:
fig = px.line(alt2, x="timestamp", y="data", markers=True, title='Cabal Massies Roda')
fig2 = px.line(altPrep2, x="timestamp", y="data", markers=True, title="Precipitations Massies Roda")
fig.show()
fig2.show()

## Analisis Precipitacions Baix Ter

In [ ]:
fig = px.line(baix1, x="timestamp", y="data", markers=True, title='Cabal Pasteral Cabal')
fig2 = px.line(baixPrep1, x="timestamp", y="data", markers=True, title="Precipitations Pasteral Cabal")
fig.show()
fig2.show()

In [ ]:
fig = px.line(baix2, x="timestamp", y="data", markers=True, title='Cabal Colomers')
fig2 = px.line(baixPrep1, x="timestamp", y="data", markers=True, title="Precipitations Colomers")
fig.show()
fig2.show()

In [ ]:
fig = px.line(baix3, x="timestamp", y="data", markers=True, title='Cabal Toroella-Montegri')
fig2 = px.line(baixPrep2, x="timestamp", y="data", markers=True, title="Precipitacions Toroella-Montegri")
fig.show()
fig2.show()

In [ ]:
def subplotsPerData(inici, final):
    
    start_date = datetime.datetime.strptime(inici, "%d/%m/%Y")
    end_date = datetime.datetime.strptime(final, "%d/%m/%Y")
    
    alt1 = selectTimespan(alt1, start_date, end_date)
    alt2 = selectTimespan(alt2, start_date, end_date)
    baix1 = selectTimespan(baix1, start_date, end_date)
    baix2 = selectTimespan(baix2, start_date, end_date)
    baix3 = selectTimespan(baix3, start_date, end_date)
    
    altPrep1 = selectTimespan(altPrep1, start_date, end_date)
    altPrep2 = selectTimespan(altPrep2, start_date, end_date)
    baixPrep1 = selectTimespan(baixPrep1, start_date, end_date)
    baixPrep2 = selectTimespan(baixPrep2, start_date, end_date)
    
    fig = make_subplots(rows=2,cols=5)
    fig.add_trace(go.Scatter(x=alt1['timestamp'],y=alt1['data']), markers=True, title='Cabal Sant Joan', row=1, col=1)
    fig.add_trace(go.Scatter(x=altPrep1['timestamp'],y=altPrep2['data']), markers=True, title='Precipitacions Sant Joan', row=1, col=2)
    
    fig.add_trace(go.Scatter(x=alt2['timestamp'],y=alt2['data']), markers=True, title='Cabal Massies Roda', row=2, col=1)
    fig.add_trace(go.Scatter(x=altPrep2['timestamp'],y=altPrep2['data']), markers=True, title='Precipitacions Massies Roda', row=2, col=2)
    
    fig.add_trace(go.Scatter(x=baix1['timestamp'],y=baix1['data']), markers=True, title='Cabal Pasteral', row=3, col=1)
    fig.add_trace(go.Scatter(x=baixPrep1['timestamp'],y=baixPrep1['data']), markers=True, title='Precipitacions Pasteral', row=3, col=2)
    
    fig.add_trace(go.Scatter(x=baix2['timestamp'],y=baix2['data']), markers=True, title='Cabal Colomers', row=4, col=1)
    fig.add_trace(go.Scatter(x=baixPrep2['timestamp'],y=baixPrep2['data']), markers=True, title='Precipitacions Colomers', row=4, col=2)
    
    fig.add_trace(go.Scatter(x=baix3['timestamp'],y=baix3['data']), markers=True, title='Cabal Torroella-Montegri', row=5, col=1)
    fig.add_trace(go.Scatter(x=baixPrep2['timestamp'],y=baixPrep2['data']), markers=True, title='Precipitacions Torroella-Montegri', row=5, col=2)
    

    